In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import data
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
import os

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import panel as pn
import joblib
import keras
import traceback
from dash.exceptions import PreventUpdate
import plotly.graph_objs as go
import traceback
import holoviews as hv
from holoviews import dim
from dash import dash_table

In [3]:
def Ion_Similator(EC, Sacramento_X2,Ion, WYT, Location, month):
    X= pd.DataFrame(columns=['EC', 'Sacramento X2', 'AN', 'BN', 'C', 'D', 'W', 'OMR', 'SJRcorridor', 'SouthDelta', 'April', 'August', 'December', 'February', 'January', 'July', 'June', 'March', 'May', 'November', 'October', 'September'])

    X.loc[0,"EC"]=(EC -50)/(3500-50)
    X.loc[0,'Sacramento X2']=Sacramento_X2/100
    X.loc[0,'AN']=1 if WYT=="AN" else 0
    X.loc[0,'BN']=1 if WYT=="BN" else 0
    X.loc[0,'C']=1 if WYT=="C" else 0
    X.loc[0,'D']=1 if WYT=="D" else 0
    X.loc[0,'W']=1 if WYT=="W" else 0

    X.loc[0,'OMR']=1 if Location=="OMR" else 0
    X.loc[0,'SJRcorridor']=1 if Location=="SJRcorridor" else 0
    X.loc[0,'SouthDelta']=1 if Location=="SouthDelta" else 0

    X.loc[0,'April']=1 if month=="April" else 0
    X.loc[0,'August']=1 if month=="August" else 0
    X.loc[0,'December']=1 if month=="December" else 0
    X.loc[0,'February']=1 if month=="February" else 0
    X.loc[0,'January']=1 if month=="January" else 0
    X.loc[0,'July']=1 if month=="July" else 0
    X.loc[0,'June']=1 if month=="June" else 0
    X.loc[0,'March']=1 if month=="March" else 0
    X.loc[0,'May']=1 if month=="May" else 0
    X.loc[0,'November']=1 if month=="November" else 0
    X.loc[0,'October']=1 if month=="October" else 0
    X.loc[0,'September']=1 if month=="September" else 0

    ANN_final= keras.models.load_model(f'Models/ANN_{Ion}.h5')
    import joblib
    MLR_final=joblib.load(filename=f'Models/MLR_{Ion}.pkl')
    RT_final =joblib.load(filename=f'Models/RT_{Ion}.pkl')
    RF_final =joblib.load(filename=f'Models/RF_{Ion}.pkl')
    GB_final =joblib.load(filename=f'Models/GB_{Ion}.pkl')

    X1=X.astype(float)

    df= pd.DataFrame(columns=['MLR', 'RT','GB', 'RF', 'ANN'])
    df.loc[0,'MLR']=round(MLR_final.predict(X)[0],2)
    df.loc[0,'RT']=round(RT_final.predict(X)[0],2)
    df.loc[0,'GB']=round(GB_final.predict(X)[0],2)
    df.loc[0,'RF']=round(RF_final.predict(X)[0],2)
    df.loc[0,'ANN']=round(ANN_final.predict(X1)[0][0].astype(float),2)


    return df



# Create Dash app
app = dash.Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1('Ions simulator Dashboard'),
    html.Div([
        html.Label('Enter your EC'),
        dcc.Input(id='num1-input', type='number', value=200),
        html.Label('Enter your Sacramento X2'),
        dcc.Input(id='num2-input', type='number', value=90),
        html.Label('Select Ion'),
        dcc.Dropdown(id='cat1-dropdown',
                     options=[{'label': 'Alkalinity', 'value': 'Alkalinity'},
                              {'label': 'Br', 'value': 'Br'},
                              {'label': 'Ca', 'value': 'Ca'},
                              {'label': 'Cl', 'value': 'Cl'},
                              {'label': 'K', 'value': 'K'},
                              {'label': 'Mg', 'value': 'Mg'},
                              {'label': 'Na', 'value': 'Na'},
                              {'label': 'SO4', 'value': 'SO4'},
                              {'label': 'TDS', 'value': 'TDS'}],
                     value='Alkalinity'),
        html.Label('Select WYT'),
        dcc.Dropdown(id='cat2-dropdown',
                     options=[{'label': 'AN', 'value': 'AN'},
                              {'label': 'BN', 'value': 'BN'},
                              {'label': 'C', 'value': 'C'},
                              {'label': 'D', 'value': 'D'},
                              {'label': 'W', 'value': 'W'}],
                     value='BN'),
        html.Label('Select Location'),
        dcc.Dropdown(id='cat3-dropdown',
                     options=[{'label': 'OMR', 'value': 'OMR'},
                              {'label': 'SJRcorridor', 'value': 'SJRcorridor'},
                              {'label': 'SouthDelta', 'value': 'SouthDelta'}],
                     value='OMR'),
        html.Label('Select Month'),
        dcc.Dropdown(id='cat4-dropdown',
                     options=[{'label': 'April', 'value': 'April'},
                              {'label': 'August', 'value': 'August'},
                              {'label': 'December', 'value': 'December'},
                              {'label': 'February', 'value': 'February'},
                              {'label': 'July', 'value': 'July'},
                              {'label': 'June', 'value': 'June'},
                              {'label': 'March', 'value': 'March'},
                              {'label': 'May', 'value': 'May'},
                              {'label': 'November', 'value': 'November'},
                              {'label': 'October', 'value': 'October'},
                              {'label': 'September', 'value': 'September'}],
                     value='June')
    ]),
    html.Div(id='output-div'),
    html.Div([
        html.H2('Definitions'),
        html.P('EC: Electrical Conductivity'),
        html.P('SacramentoX2: The exact location of the Sacramento X2 point is determined by the California Department of Water Resources (DWR) based on the specific hydraulic conditions and water flows in the Sacramento River. The DWR uses a combination of hydrological models, flow measurements, and other data to determine the location of the Sacramento X2 point.'),
        html.P('WYT: Water Year Type that includes the following categories: 1- Wet (W), 2- Critical (C), 3- Dry (D), 4- Above-Normal (AN), 5- Below-Normal (BN)'),
        html.P('Location: Monitoring regions that includes: 1- Old-Middle River (OMR), 2- San Joaquin River Corridor (SJRcorridor), and 3- South Delta (SouthDelta)'),
        html.P('Month: Twelve months in a year'),
        html.P('Machine Learning Models: 1- Multiple Linear Regression (MLR), 2- Regression Trees (RT), 3- Gradient Boosting (GB), and 4- Artificial Neural Network (ANN)')
    ], style={'border': '1px solid #ddd', 'padding': '10px', 'margin-top': '20px'})
])


# Define callback to update output when inputs are changed
@app.callback(Output('output-div', 'children'),
              [Input('num1-input', 'value'),
               Input('num2-input', 'value'),
               Input('cat1-dropdown', 'value'),
               Input('cat2-dropdown', 'value'),
               Input('cat3-dropdown', 'value'),
               Input('cat4-dropdown', 'value')])
def update_output(EC, Sacramento_X2,Ion, WYT, Location, month):
    result_df = Ion_Similator(EC, Sacramento_X2,Ion, WYT, Location, month)
    ion_label = Ion + ' [mg/L]'
    return html.Div([
        html.H2('Results of ion simulators:'),
        html.Br(),
        dcc.Graph(
            figure={
                'data': [{'x': result_df.columns, 'y': result_df.iloc[0], 'type': 'bar','text': result_df.iloc[0].apply(lambda x: round(x, 2)), # add text to show values
                    'textposition': 'auto' # specify text position
                          }],
                'layout': {
                    'height': 500,
                    'width': 800,
                    'xaxis': {'title': 'Machine Learning models'},
                    'yaxis': {'title': ion_label}
                }
            }
        )
    ])
# Run the app
if __name__ == '__main__':
    try:
        app.run_server(debug=False)
    except Exception as e:
        print(e)
        traceback.print_exc()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [08/Mar/2023 11:35:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 11:35:10] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 11:35:10] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 11:35:10] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -


1/1 [==============================] - 0s 92ms/step


127.0.0.1 - - [08/Mar/2023 11:35:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2023 11:35:11] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [08/Mar/2023 11:35:11] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
